<a href="https://colab.research.google.com/github/Santi-Guti/NB_IA_AP/blob/main/CPU_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> # **1.Lectura de datos**




## Lectura del set de datos (almacenado en google drive)

In [ ]:
import numpy as np
import os
import gzip

def load_mnist(ruta, tipo='train'):

    ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' % tipo)
    ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)
    
    with gzip.open(ruta_categorias, 'rb') as rut_cat:
        etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8)

    with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
        imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas), 784)

    return imagenes, etiquetas

## Acceso a datos almecenados en Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

ruta = '/content/gdrive/MyDrive/Colab Notebooks/fashion_mnist_data'

X_train, Y_train = load_mnist(ruta, tipo='train')
X_test, Y_test = load_mnist(ruta, tipo='test')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## $Reshape$ de los datos para grantizar que sean matrices (no vectores)

In [ ]:
print(X_train.shape)
print(Y_train.shape)

(60000, 784)
(60000,)


In [ ]:
X_train = X_train[0:59904,:]
X_test = X_test[0:9984,:]
Y_train = Y_train[0:59904]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train,(59904,28,28,1))
X_test = np.reshape(X_test,(9984,28,28,1))

> # **2.Creación del modelo con TensorFlow 2 + Keras**

In [ ]:
%tensorflow_version 2.x   # Para garantizar que la versión 2.x sea importada
import tensorflow as tf
print('Versión de TensorFlow: ' + tf.__version__)

tf.random.set_seed(200)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))
model.summary()

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x   # Para garantizar que la versión 2.x sea importada`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
Versión de TensorFlow: 2.4.1
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_7 (Batch (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 64)        1664      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 14

> # **3.Entrenamiento con CPU**

In [ ]:
import timeit

def entrenamiento_cpu():
  with tf.device('/cpu:0'):
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1)
  
  return None

cpu_time = timeit.timeit('entrenamiento_cpu()', number=1, setup='from __main__ import entrenamiento_cpu')

Epoch 1/2
468/468 [==============================] - 667s 1s/step - loss: 1.0169 - accuracy: 0.7078 - val_loss: 0.4264 - val_accuracy: 0.8551
Epoch 2/2
468/468 [==============================] - 666s 1s/step - loss: 0.4105 - accuracy: 0.8562 - val_loss: 0.3127 - val_accuracy: 0.8922


In [ ]:
print('Tiempo de entrenamiento: ' + str(cpu_time) + ' segundos')

Tiempo de entrenamiento: 1332.837365464 segundos
